Reading the GRBL instructions then the Marlin lead to a rabbit's hole of G-Code.

http://reprap.org/wiki/G-code

In [1]:
flavors = ["GRBL", "Marlin", "RS274"]

[GRBL](https://github.com/gnea/grbl/wiki) - An open source, embedded, high performance g-code-parser and CNC milling controller written in optimized C that will run on a straight Arduino

[Marlin](http://marlinfw.org/) - Open Source 3D Printer Firmware

[NIST RS274NGC Interpreter - Version 3](http://ws680.nist.gov/publication/get_pdf.cfm?pub_id=823374) - The real deal.

        Intelligent Systems Division
        National Institute of Standards and Technology
        Technology Administration
        U.S. Department of Commerce
        Gaithersburg, Maryland 20899

Starting point.

### V0.01

In [2]:
class GCode(object):
    pass

class GRBL(GCode):
    pass

class Marlin(GCode):
    pass

class RS274(GCode):
    pass

Compare & Contrast.

In [3]:
x=dict()
x["a"]=1
x["2"]=2

In [4]:
x.keys()

dict_keys(['a', '2'])

In [5]:
x.values()

dict_values([1, 2])

### V0.02

In [6]:
class GCode(object):
    def do(self, cmd, **kwargs):
        args = list()
        for key, value in kwargs.items():
            args.append("{key}{value}".format(key=key, value=value))
        
        print("{cmd} {args}".format(cmd=cmd, args=" ".join(args)))

In [7]:
machine = GCode()
machine.do("G0", X=0, Y=0, Z=0)

G0 Y0 Z0 X0


In [8]:
machine = GCode()
machine.do("G1", X=10, Y=10, Z=10)

G1 Y10 Z10 X10


In [9]:
machine = GCode()
# Draw a 10x10 Square.
machine.do("G1", X=0, Y=0)
machine.do("G1", X=10, Y=0)
machine.do("G1", X=10, Y=10)
machine.do("G1", X=0, Y=10)
machine.do("G1", X=0, Y=0)

G1 Y0 X0
G1 Y0 X10
G1 Y10 X10
G1 Y10 X0
G1 Y0 X0


### V0.03

In [10]:
class GCode(object):
    def do(self, cmd, **kwargs):
        args = list()
        for key, value in kwargs.items():
            args.append("{key}{value}".format(key=key, value=value))
        
        print("{cmd} {args}".format(cmd=cmd, args=" ".join(args)))
        
    def G0(self, **kwargs):
        self.do("G0", **kwargs)
    def G1(self, **kwargs):
        self.do("G1", **kwargs)
    def G2(self, **kwargs):
        self.do("G2", **kwargs)
    def G3(self, **kwargs):
        self.do("G3", **kwargs)

In [11]:
machine = GCode()

machine.G1(X=0, Y=10)

G1 Y10 X0


In [12]:
machine.G2(X=0, Y=10)

G2 Y10 X0


#### V0.031

That was going to get old very fast.

In [13]:
# Good core to start with.
commands = ["G0", "G1", "G20", "G21", "G28"]
# Add in tool codes.
for code in [0, 1, 2, 3, 4, 5, 6]:
    commands.append("M{code}".format(code=code))
# Commands to implement.
print(commands)

['G0', 'G1', 'G20', 'G21', 'G28', 'M0', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6']


In [14]:
class GCode(object):
    def do(self, cmd, **kwargs):
        args = list()
        for key, value in kwargs.items():
            args.append("{key}{value}".format(key=key, value=value))
        
        print("{cmd} {args}".format(cmd=cmd, args=" ".join(args)))

def cmd_factory(cmd):
    def cmd_fcn(self, **kwargs):
        self.do(cmd, **kwargs)
    return cmd_fcn

In [15]:
machine = GCode()
machine

In [16]:
G0 = cmd_factory("G0")
G0

<function __main__.cmd_factory.<locals>.cmd_fcn>

In [17]:
G0(machine, X=0, Y=0)

G0 Y0 X0


In [18]:
class GCode(object):
    def do(self, cmd, **kwargs):
        args = list()
        for key, value in kwargs.items():
            args.append("{key}{value}".format(key=key, value=value))
        
        print("{cmd} {args}".format(cmd=cmd, args=" ".join(args)))

def cmd_factory(cmd):
    def cmd_fcn(self, **kwargs):
        self.do(cmd, **kwargs)
    return cmd_fcn

for command in commands:
    print(command)
    setattr(GCode, command, cmd_factory(command))

G0
G1
G20
G21
G28
M0
M1
M2
M3
M4
M5
M6


In [19]:
machine = GCode()

In [20]:
machine.G0(X=0)

G0 X0


In [21]:
machine.G1(X=0)

G1 X0


In [22]:
machine.M3(S=100)

M3 S100


In [23]:
machine.G0(X=0)

G0 X0


In [24]:
for X in range(0, 100, 10):
    machine.G0(X=X)

G0 X0
G0 X10
G0 X20
G0 X30
G0 X40
G0 X50
G0 X60
G0 X70
G0 X80
G0 X90


#### V0.05

In [89]:
class GCode(object):
    def __init__(self):
        self.linenumbering = False
        self.buffer = list()
        
    @property
    def code(self):
        if self.linenumbering:
            pass
        return "\n".join(self.buffer)
        
    def save(self, filename):
        pass
    
    def send(self):
        pass
    
    def __str__(self):
        return self.code

    def __repr__(self):
        return "<GCode>[cmds={}]".format(len(self.buffer))
    
    def _repr_html_(self):
        html = list()
        for cmd_line in self.buffer:
            cmd, *args = cmd_line.split(" ")
            html_line = "<b>{cmd}</b> <i>{args}</i>".format(cmd=cmd, args=" ".join(args))
            html.append(html_line)
        return "<br>\n".join(html)
    

    def cmd_factory(cmd, doc=None):
    def cmd_fcn(self, **kwargs):
        args = list()
        for key, value in kwargs.items():
            args.append("{key}{value}".format(key=key, value=value))
        
        cmd_str = "{cmd} {args}".format(cmd=cmd, args=" ".join(args))
        self.buffer.append(cmd_str)  
    return cmd_fcn       


for command in commands:
    setattr(GCode, command, cmd_factory(command))
                 
                 
machine = GCode()
machine.G0(X=0)

In [90]:
machine.buffer

['G0 X0']

In [91]:
machine

<GCode>[cmds=1]

In [92]:
machine.G0(X=10)

In [93]:
print(machine)

G0 X0
G0 X10


In [94]:
import IPython.display

In [95]:
IPython.display.display_html(machine)

G0 X0 
 G0 X10

In [96]:
IPython.display.display_json(machine)

Draw a 100x100mm Rectangle.

In [27]:
machine = GCode()
